In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
from einops import rearrange, repeat
from transformers import CLIPTextModel, CLIPTokenizer

import sys
sys.path.append('/Users/xiaodong/新项目/diffusers/src')
from diffusers import UNetSpatioTemporalConditionModel_Action
from diffusers import AutoencoderKL, DDPMScheduler, StableDiffusionPipeline, UNet2DConditionModel
from diffusers import ActionVideoDiffusionPipeline

In [2]:
base_model = UNet2DConditionModel(cross_attention_dim=768, in_channels=4)

In [3]:
base_state_dicts = base_model.state_dict()
del base_model

In [4]:
unet = UNetSpatioTemporalConditionModel_Action(cross_attention_dim=768, in_channels=4)

add image context parameters
spatial out_channels: 320
add image context parameters
spatial out_channels: 640
add image context parameters
spatial out_channels: 1280
add image context parameters
spatial out_channels: 1280
add image context parameters
spatial out_channels: 1280


In [ ]:
# down_blocks.0.attentions.0.transformer_blocks.0.norm1.weight
# down_blocks.0.attentions.0.transformer_blocks.0.norm1.weight

# down_blocks.0.resnets.0.spatial_res_block.conv1.weight
# down_blocks.0.resnets.0.conv1.weight

# down_blocks.0.resnets.0.time_emb_proj.weight
# down_blocks.0.resnets.0.spatial_res_block.time_emb_proj.weight

In [19]:
k = "down_blocks.0.resnets.0.time_emb_proj.weight"
strs = k.split('.')
new_k = '.'.join(strs[:4]) + '.spatial_res_block.' + '.'.join(strs[-2:])
new_k
# down_blocks.0.resnets.0.spatial_res_block.time_emb_proj.weight

'down_blocks.0.resnets.0.spatial_res_block.time_emb_proj.weight'

In [13]:
# 更新的版本
from collections import OrderedDict
new_state_dicts = OrderedDict()
for k, v in base_state_dicts.items():
    
    if 'resnets' in k:
        strs = k.split('.')
        new_k = '.'.join(strs[:4]) + '.spatial_res_block.' + '.'.join(strs[-2:])
        new_state_dicts[new_k] = v
    else:
        new_state_dicts[k] = v

In [14]:
miss_keys, ignore_keys = unet.load_state_dict(new_state_dicts, strict=False)

In [5]:
# 修改的版本
from collections import OrderedDict
new_state_dicts = OrderedDict()
for k, v in base_state_dicts.items():
    if 'mid_block' in k and 'resnets' in k:
        strs = k.split('.')
        new_k = '.'.join(strs[:3]) + '.spatial_res_block.' + '.'.join(strs[-2:])
        new_state_dicts[new_k] = v
    elif 'resnets' in k and 'temporal_res_blocks' not in k:
        strs = k.split('.')
        new_k = '.'.join(strs[:4]) + '.spatial_res_block.' + '.'.join(strs[-2:])
        new_state_dicts[new_k] = v
    else:
        new_state_dicts[k] = v

In [6]:
miss_keys, ignore_keys = unet.load_state_dict(new_state_dicts, strict=False)

In [15]:
len(miss_keys) # 816
## 错误的版本

816

In [8]:
len(miss_keys) # 796一般是正解

796

In [7]:
ignore_keys

[]

In [16]:
miss_keys

['add_embedding.linear_1.weight',
 'add_embedding.linear_1.bias',
 'add_embedding.linear_2.weight',
 'add_embedding.linear_2.bias',
 'down_blocks.0.attentions.0.temporal_transformer_blocks.0.norm_in.weight',
 'down_blocks.0.attentions.0.temporal_transformer_blocks.0.norm_in.bias',
 'down_blocks.0.attentions.0.temporal_transformer_blocks.0.ff_in.net.0.proj.weight',
 'down_blocks.0.attentions.0.temporal_transformer_blocks.0.ff_in.net.0.proj.bias',
 'down_blocks.0.attentions.0.temporal_transformer_blocks.0.ff_in.net.2.weight',
 'down_blocks.0.attentions.0.temporal_transformer_blocks.0.ff_in.net.2.bias',
 'down_blocks.0.attentions.0.temporal_transformer_blocks.0.norm1.weight',
 'down_blocks.0.attentions.0.temporal_transformer_blocks.0.norm1.bias',
 'down_blocks.0.attentions.0.temporal_transformer_blocks.0.attn1.to_q.weight',
 'down_blocks.0.attentions.0.temporal_transformer_blocks.0.attn1.to_k.weight',
 'down_blocks.0.attentions.0.temporal_transformer_blocks.0.attn1.to_v.weight',
 'down_bl

In [9]:
unet_keys = unet.state_dict().keys()

In [10]:
unet_keys = list(unet_keys)

In [11]:
set(unet_keys) - set(miss_keys)

{'conv_in.bias',
 'conv_in.weight',
 'conv_norm_out.bias',
 'conv_norm_out.weight',
 'conv_out.bias',
 'conv_out.weight',
 'down_blocks.0.attentions.0.norm.bias',
 'down_blocks.0.attentions.0.norm.weight',
 'down_blocks.0.attentions.0.proj_in.bias',
 'down_blocks.0.attentions.0.proj_in.weight',
 'down_blocks.0.attentions.0.proj_out.bias',
 'down_blocks.0.attentions.0.proj_out.weight',
 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_k.weight',
 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.0.bias',
 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.0.weight',
 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.weight',
 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_v.weight',
 'down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_k.weight',
 'down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out.0.bias',
 'down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out.0.weight',
 'down_blocks.0.attentions.0.transformer

In [13]:
pwd

'/Users/xiaodong/新项目/diffusers/examples/text_to_video'